<a href="https://colab.research.google.com/github/trehansalil/Py_Projects/blob/master/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pymongo[srv] scikit-multilearn transformers emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 454.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 868.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 778.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 22.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 18.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.0 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.3 MB/s eta 0:00:00


In [54]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime, timezone
import os
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW


# Requires the PyMongo package.
# https://api.mongodb.com/python/current

# from google.colab import userdata
MONGO_DB_URI = os.environ['MONGO_DB_URI']

client = MongoClient(MONGO_DB_URI)
filter={
    'updated_at': {
        '$gte': datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc),
        '$lt': datetime(2025, 1, 1, 0, 0, 0, tzinfo=timezone.utc)
    },
    'category': {
        '$ne': []
    }
}
sort=list({
    'updated_at': -1
}.items())

result = client['inshorts_db']['news_data'].find(
  filter=filter,
  sort=sort
)

df = pd.DataFrame.from_dict(result)

irrelevant_categories = [i for i in df['category'].explode().value_counts().loc[df['category'].explode().value_counts()<1000].index]
relevant_categories = [i for i in df['category'].explode().value_counts().loc[df['category'].explode().value_counts()>=1000].index]

print(irrelevant_categories)

df['category'] = df['category'].apply(lambda x: [i for i in x if i not in irrelevant_categories])

df.drop(index=df.loc[df['category'].apply(lambda x: x==[])].index, inplace=True)

df = df.reset_index(drop=True)

df.head()

['IPL', 'Lifestyle', 'EXPLAINERS', 'Feel_Good_Stories', 'experiment', 'Israel-Hamas_War', 'cricket', 'Russia-Ukraine_Conflict', 'city', 'crime', 'bollywood', 'Coronavirus', 'cryptocurrency', 'football', 'arts_and_entertainment', 'Union_Budget_2023-24', 'Union_Budget_2024', 'environment', 'ODI_World_Cup_2023']


,_id,inshorts_url,author,category,created_at,date,datetime,day,headlines,image_url,original_source,read_more,relevancy_tags,text,updated_at
0,6648e7804af552158722f350,https://inshorts.com/en/news/situation-is-calm...,Ankush Verma,"[world, national]",2024-05-19 18:59:01.505,2024-05-18,2024-05-18 17:01:51,Saturday,Situation is calm and under control: Kyrgyzsta...,https://nis-gs.pix.in/inshorts/images/v1/varia...,Hindustan Times,https://www.hindustantimes.com/world-news/kyrg...,"[world, national]",Kyrgyzstan has released a statement after Indi...,2024-05-19 18:59:01.505
1,664959b34af5521587239520,https://inshorts.com/en/news/carryminati-apolo...,Daisy Mowke,[entertainment],2024-05-19 18:59:01.228,2024-05-18,2024-05-18 17:22:42,Saturday,CarryMinati apologises to influencer Rajat Dal...,https://nis-gs.pix.in/inshorts/images/v1/varia...,Free Press Journal,https://www.freepressjournal.in/amp/india/carr...,[celebrities],CarryMinati apologised and deleted a part of a...,2024-05-19 18:59:01.228
2,664959b24af5521587239493,https://inshorts.com/en/news/more-northern-lig...,Daisy Mowke,[science],2024-05-19 18:59:00.952,2024-05-18,2024-05-18 17:34:16,Saturday,More Northern Lights expected as sunspot clust...,https://nis-gs.pix.in/inshorts/images/v1/varia...,News18,https://www.news18.com/amp/world/more-northern...,[science],"The vast sunspot cluster, which released energ...",2024-05-19 18:59:00.952
3,6648f5e74af552158743ca4d,https://inshorts.com/en/news/rcb-knock-csk-out...,Anmol Sharma,[sports],2024-05-19 18:59:00.675,2024-05-18,2024-05-18 18:36:12,Saturday,"RCB knock CSK out of IPL 2024, reach playoffs ...",https://nis-gs.pix.in/inshorts/images/v1/varia...,ESPNcricinfo,https://www.espncricinfo.com/series/indian-pre...,[cricket],RCB defeated CSK by 27 runs at M Chinnaswamy S...,2024-05-19 18:59:00.675
4,664967bd4af552158742f1f2,https://inshorts.com/en/news/visakhapatnam-pol...,System User,[national],2024-05-19 18:59:00.398,2024-05-19,2024-05-19 01:36:16,Sunday,"Visakhapatnam Police bust cyber ring, arrest 3...",https://nis-gs.pix.in/inshorts/images/v1/varia...,Newskarnataka,https://newskarnataka.com/india/visakhapatnam-...,[],Visakhapatnam Police dismantled a cybercrime r...,2024-05-19 18:59:00.398


In [55]:
df['category'].explode().value_counts()

category
national               4983
business               3500
sports                 2972
entertainment          2529
world                  2435
politics               2273
technology             1652
miscellaneous          1563
hatke                  1452
startup                1344
travel                 1107
fashion                1106
science                1105
LOK_SABHA_ELECTIONS    1079
Health___Fitness       1072
automobile             1034
education              1005
Name: count, dtype: int64

In [56]:
df.columns

Index(['_id', 'inshorts_url', 'author', 'category', 'created_at', 'date',
       'datetime', 'day', 'headlines', 'image_url', 'original_source',
       'read_more', 'relevancy_tags', 'text', 'updated_at'],
      dtype='object')

In [57]:
df['original_source'].value_counts()

original_source
Free Press Journal    1593
Times Now             1209
Hindustan Times       1088
News Karnataka         636
SheThePeople           631
                      ... 
NDTV profit              1
AAP                      1
XYXX Apparels            1
Evening Standard         1
mini.in                  1
Name: count, Length: 566, dtype: int64

In [58]:
df.drop(columns=['inshorts_url', 'author', 'created_at', 'updated_at', 
                 'datetime', 'date', 'day', 'original_source', 
                 'read_more', 'relevancy_tags'],inplace=True)

In [59]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [60]:
from bs4 import BeautifulSoup
import re
import emoji
import string

def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    # text = BeautifulSoup(text, 'lxml').get_text()
    # text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    # text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    # text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

df['text'] = df['text'].apply(text_preprocessing_pipeline)

df_labels = df['category'].apply(lambda x: ",".join(x)).str.get_dummies(sep=',')

df = df.join(df_labels)

In [61]:
len(irrelevant_categories)

19

In [62]:
len(relevant_categories)

17

In [67]:
target_cols = [i for i in df_labels.columns]

In [64]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [65]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [68]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [73]:
df_test = df.loc[df.index%5 == 0]
df_train = df.loc[df.index%5 != 0]

train_loader = DataLoader(df_train, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(df_test, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

/home/codespace/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [74]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,5)
    
    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

model = BERTClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTClass(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [75]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [76]:
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)


/home/codespace/.python/current/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [77]:
def train(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [78]:
for epoch in range(EPOCHS):
    train(epoch)

/home/codespace/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 9676

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/codespace/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", 

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MultiLabelBinarizer

import tensorflow as tf

class HammingLoss(tf.keras.metrics.Metric):
    def __init__(self, name='hamming_loss', threshold=0.5, mode='multilabel', **kwargs):
        super(HammingLoss, self).__init__(name=name, **kwargs)
        self.threshold = threshold
        self.mode = mode
        self.total_loss = self.add_weight(name='total_loss', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        if self.mode == 'multilabel':
            y_pred = tf.cast(y_pred > self.threshold, tf.int32)
        elif self.mode == 'multiclassification':
            y_pred = tf.argmax(y_pred, axis=1)
            y_true = tf.argmax(y_true, axis=1)

        hamming_distance = tf.reduce_mean(tf.cast(y_true != y_pred, tf.float32))

        self.total_loss.assign_add(hamming_distance)
        self.count.assign_add(1)

    def result(self):
        return self.total_loss / self.count

    def reset_states(self):
        self.total_loss.assign(0)
        self.count.assign(0)



# Text Preprocessing
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text'])
X_text = tokenizer.texts_to_sequences(df['text'])
X_text = pad_sequences(X_text, padding='post')

# Category Preprocessing
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['category'])

# Convert category labels to one-hot encoding
y = np.array(y)


model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(mlb.classes_), activation='sigmoid'))

metric = HammingLoss(mode='multilabel', threshold=0.8)

optimizer = tf.keras.optimizers.AdamW(
                learning_rate=0.001,
                weight_decay=0.004,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-07,
                amsgrad=False,
                clipnorm=None,
                clipvalue=None,
                global_clipnorm=None,
                use_ema=False,
                ema_momentum=0.99,
                ema_overwrite_frequency=None,
                loss_scale_factor=None,
                gradient_accumulation_steps=None,
                name='adamw',
            )

# Compiling the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', metric])

# Training the model
history = model.fit(X_text, y, epochs=10, batch_size=16, validation_split=0.2)


2024-05-19 18:25:17.806572: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-19 18:25:17.810026: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-19 18:25:17.842307: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 18:25:20.242522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/10


TypeError: Input 'y' of 'NotEqual' Op has type int32 that does not match type int64 of argument 'x'.